# Unit 12: Multivariate Analysis - Practical

## 🎯 Learning Objectives

In this practical notebook, you'll learn to:
1. Create and interpret pair plots (scatterplot matrices)
2. Build correlation heatmaps for multiple variables
3. Use parallel coordinates plots
4. Create 3D scatter plots and bubble charts
5. Build facet grids for conditional analysis
6. Detect and visualize interaction effects
7. Identify multivariate outliers
8. Use dimensionality reduction (PCA, t-SNE)

## 📚 Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully!")

---

## Part 1: Pair Plots (Scatterplot Matrices) 📊

**Goal:** Visualize all pairwise relationships at once

**Dataset:** Iris flowers (classic multivariate dataset)

In [ ]:
# Load Iris dataset
iris = sns.load_dataset('iris')

print("Iris Dataset:")
print(iris.head(10))
print(f"\nShape: {iris.shape}")
print(f"\nFeatures: {list(iris.columns)}")
print(f"\nSpecies: {iris['species'].unique()}")

In [ ]:
# Create basic pair plot
print("📊 BASIC PAIR PLOT")
print("="*60)

sns.pairplot(iris)
plt.suptitle('Iris Dataset - All Pairwise Relationships', y=1.02, fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n🔍 What to observe:")
print("  ✓ Diagonal: Distribution of each feature (histograms)")
print("  ✓ Off-diagonal: Scatter plots of feature pairs")
print("  ✓ Look for linear relationships, clusters, outliers")

In [ ]:
# Create pair plot with species coloring
print("\n📊 PAIR PLOT WITH SPECIES COLORING")
print("="*60)

sns.pairplot(iris, hue='species', palette='Set1', diag_kind='kde', markers=['o', 's', 'D'])
plt.suptitle('Iris Dataset - Colored by Species', y=1.02, fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n🔍 Key Observations:")
print("  ✓ Setosa (red) clearly separated on petal measurements")
print("  ✓ Versicolor (green) and Virginica (blue) overlap on sepal measurements")
print("  ✓ petal_length vs petal_width shows best separation")
print("  ✓ All species have roughly normal distributions")

In [ ]:
# Focused pair plot (subset of variables)
print("\n📊 FOCUSED PAIR PLOT (Petal measurements only)")
print("="*60)

sns.pairplot(iris, vars=['petal_length', 'petal_width'], hue='species', palette='Set2', height=3.5)
plt.suptitle('Iris Petal Measurements', y=1.02, fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 Insight:")
print("  - Strong positive correlation between petal length and width")
print("  - Three distinct clusters (species)")
print("  - These two features alone could classify species well")

---

## Part 2: Correlation Heatmaps 🔥

**Goal:** See all correlations at a glance

**Dataset:** House prices with multiple features

In [ ]:
# Create house price dataset
np.random.seed(42)
n_houses = 300

house_data = pd.DataFrame({
    'size_sqft': np.random.randint(1000, 4000, n_houses),
    'bedrooms': np.random.randint(2, 6, n_houses),
    'bathrooms': np.random.uniform(1, 4, n_houses),
    'age_years': np.random.randint(0, 40, n_houses),
    'lot_size': np.random.randint(3000, 15000, n_houses),
    'garage_spaces': np.random.randint(0, 4, n_houses),
    'distance_downtown_km': np.random.uniform(2, 25, n_houses)
})

# Generate price with realistic relationships
house_data['price'] = (
    200 * house_data['size_sqft'] +
    25000 * house_data['bedrooms'] +
    30000 * house_data['bathrooms'] +
    -4000 * house_data['age_years'] +
    15 * house_data['lot_size'] +
    12000 * house_data['garage_spaces'] +
    -8000 * house_data['distance_downtown_km'] +
    np.random.normal(0, 40000, n_houses) +
    100000
)

print("House Price Dataset:")
print(house_data.head())
print(f"\nShape: {house_data.shape}")
print(f"\nFeatures: {list(house_data.columns)}")

In [ ]:
# Calculate correlation matrix
print("📊 CORRELATION MATRIX")
print("="*60)

correlation_matrix = house_data.corr()

print("\nCorrelation Matrix:")
print(correlation_matrix.round(2))

In [ ]:
# Create correlation heatmap
print("\n🔥 CORRELATION HEATMAP")
print("="*60)

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={'label': 'Correlation'})
plt.title('House Features Correlation Heatmap', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n🔍 Reading the Heatmap:")
print("  Red: Positive correlation (variables move together)")
print("  Blue: Negative correlation (variables move oppositely)")
print("  White: No correlation")
print("  Diagonal: Always 1.0 (perfect self-correlation)")

In [ ]:
# Identify strong correlations
print("\n📈 STRONG CORRELATIONS WITH PRICE")
print("="*60)

price_correlations = correlation_matrix['price'].drop('price').sort_values(ascending=False)

print("\nFeatures ranked by correlation with price:")
print("-" * 60)
for feature, corr in price_correlations.items():
    strength = "Strong" if abs(corr) > 0.7 else "Moderate" if abs(corr) > 0.4 else "Weak"
    direction = "positive" if corr > 0 else "negative"
    stars = "⭐" * min(5, int(abs(corr) * 6))
    print(f"{feature:25s}: {corr:+.3f} ({strength} {direction}) {stars}")

print("\n💡 Insights:")
top_feature = price_correlations.index[0]
print(f"  - Most important: {top_feature} (r = {price_correlations.iloc[0]:.3f})")
print(f"  - Least important: {price_correlations.index[-1]} (r = {price_correlations.iloc[-1]:.3f})")

In [ ]:
# Detect multicollinearity (features correlated with each other)
print("\n⚠️  MULTICOLLINEARITY DETECTION")
print("="*60)

# Find pairs with high correlation (excluding diagonal and duplicates)
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            high_corr_pairs.append((
                correlation_matrix.columns[i],
                correlation_matrix.columns[j],
                correlation_matrix.iloc[i, j]
            ))

if high_corr_pairs:
    print("\nHighly correlated feature pairs (|r| > 0.7):")
    print("-" * 60)
    for feat1, feat2, corr in high_corr_pairs:
        print(f"  {feat1} <-> {feat2}: r = {corr:.3f}")
    print("\n💡 Action: Consider removing one feature from each pair (redundancy)")
else:
    print("\n✅ No strong multicollinearity detected (all pairs |r| < 0.7)")
    print("   Features provide independent information")

---

## Part 3: Parallel Coordinates Plot 📏

**Goal:** Visualize patterns across multiple variables

**Dataset:** Palmer Penguins

In [ ]:
# Load penguins dataset
penguins = sns.load_dataset('penguins').dropna()

print("Palmer Penguins Dataset:")
print(penguins.head())
print(f"\nShape: {penguins.shape}")
print(f"\nSpecies: {penguins['species'].unique()}")
print(f"\nIslands: {penguins['island'].unique()}")

In [ ]:
# Prepare data for parallel coordinates (normalize)
print("📏 PARALLEL COORDINATES PLOT")
print("="*60)

# Select numerical columns
numerical_cols = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']

# Normalize data
scaler = StandardScaler()
penguins_normalized = penguins.copy()
penguins_normalized[numerical_cols] = scaler.fit_transform(penguins[numerical_cols])

print("\n✅ Data normalized (z-score scaling)")
print("   All features now on comparable scales")

In [ ]:
# Create parallel coordinates plot
from pandas.plotting import parallel_coordinates

plt.figure(figsize=(14, 6))
parallel_coordinates(penguins_normalized[numerical_cols + ['species']], 
                    'species', 
                    color=['#E74C3C', '#3498DB', '#2ECC71'],
                    alpha=0.4)
plt.title('Penguin Species - Parallel Coordinates Plot', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Features', fontsize=12)
plt.ylabel('Normalized Value', fontsize=12)
plt.legend(title='Species', loc='upper right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n🔍 How to Read:")
print("  - Each line = one penguin")
print("  - Each vertical axis = one feature")
print("  - Line height = feature value (normalized)")
print("  - Line color = species")

print("\n💡 Observations:")
print("  ✓ Adelie (red): Shorter bills, shorter flippers, lighter")
print("  ✓ Gentoo (green): Longer flippers, heavier, shallower bills")
print("  ✓ Chinstrap (blue): Longer bills, medium size")
print("  ✓ Clear separation on flipper_length_mm and body_mass_g")

---

## Part 4: 3D Scatter Plot and Bubble Chart 🫧

**Goal:** Visualize 3-4 variables simultaneously

In [ ]:
# Create 3D scatter plot
print("📊 3D SCATTER PLOT")
print("="*60)

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Color by species
colors_map = {'Adelie': 'red', 'Gentoo': 'green', 'Chinstrap': 'blue'}
colors = penguins['species'].map(colors_map)

scatter = ax.scatter(penguins['bill_length_mm'], 
                    penguins['bill_depth_mm'], 
                    penguins['flipper_length_mm'],
                    c=colors, 
                    s=50, 
                    alpha=0.6)

ax.set_xlabel('Bill Length (mm)', fontsize=11, labelpad=10)
ax.set_ylabel('Bill Depth (mm)', fontsize=11, labelpad=10)
ax.set_zlabel('Flipper Length (mm)', fontsize=11, labelpad=10)
ax.set_title('Penguin Measurements - 3D View', fontsize=14, fontweight='bold', pad=20)

# Create legend
from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], marker='o', color='w', markerfacecolor=c, markersize=8, label=s)
                  for s, c in colors_map.items()]
ax.legend(handles=legend_elements, title='Species')

plt.tight_layout()
plt.show()

print("\n💡 Note: In interactive mode, you can rotate this plot!")
print("   3D plots work best when interactive")

In [ ]:
# Create bubble chart (4 variables)
print("\n🫧 BUBBLE CHART (4 variables)")
print("="*60)

plt.figure(figsize=(12, 8))

# 4 variables: x, y, size, color
for species in penguins['species'].unique():
    data = penguins[penguins['species'] == species]
    plt.scatter(data['bill_length_mm'], 
               data['flipper_length_mm'],
               s=data['body_mass_g'] / 20,  # Size by body mass
               c=data['bill_depth_mm'],  # Color by bill depth
               cmap='viridis',
               alpha=0.6,
               label=species,
               edgecolors='black',
               linewidth=0.5)

plt.xlabel('Bill Length (mm)', fontsize=12)
plt.ylabel('Flipper Length (mm)', fontsize=12)
plt.title('Penguin Measurements - Bubble Chart\n(Size=Body Mass, Color=Bill Depth)', 
         fontsize=14, fontweight='bold')
plt.legend(title='Species')
plt.colorbar(label='Bill Depth (mm)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n🔍 Reading the Bubble Chart:")
print("  X-axis: Bill length")
print("  Y-axis: Flipper length")
print("  Bubble size: Body mass (larger = heavier)")
print("  Bubble color: Bill depth (darker = deeper bill)")

print("\n💡 Insight:")
print("  - Gentoo penguins: Large bubbles (heavy), long flippers")
print("  - Adelie penguins: Medium bubbles, shorter bills, deeper bills (darker)")
print("  - All 4 variables help distinguish species!")

---

## Part 5: Facet Grids 🔲

**Goal:** Compare relationships across multiple categories

**Dataset:** Tips dataset

In [ ]:
# Load tips dataset
tips = sns.load_dataset('tips')

print("Tips Dataset:")
print(tips.head(10))
print(f"\nShape: {tips.shape}")
print(f"\nColumns: {list(tips.columns)}")
print(f"\nCategories:")
print(f"  Time: {tips['time'].unique()}")
print(f"  Day: {tips['day'].unique()}")
print(f"  Sex: {tips['sex'].unique()}")
print(f"  Smoker: {tips['smoker'].unique()}")

In [ ]:
# Single facet grid (by time)
print("📊 SINGLE FACET GRID (by time of day)")
print("="*60)

g = sns.FacetGrid(tips, col='time', height=5, aspect=1.2)
g.map(sns.scatterplot, 'total_bill', 'tip', alpha=0.6)
g.set_axis_labels('Total Bill ($)', 'Tip ($)')
g.fig.suptitle('Bill vs Tip by Time of Day', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n💡 Observation:")
print("  - Dinner: Higher bills, higher tips")
print("  - Lunch: Lower bills, lower tips")
print("  - Relationship (bill-tip) similar in both times")

In [ ]:
# Two-way facet grid (time × smoker)
print("\n📊 TWO-WAY FACET GRID (time × smoker)")
print("="*60)

g = sns.FacetGrid(tips, col='time', row='smoker', height=4, aspect=1.2)
g.map(sns.scatterplot, 'total_bill', 'tip', alpha=0.6)
g.set_axis_labels('Total Bill ($)', 'Tip ($)')
g.fig.suptitle('Bill vs Tip by Time and Smoking Status', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

print("\n💡 Observations:")
print("  - Consistent positive relationship in all groups")
print("  - Dinner-Smoker: Some high bills")
print("  - Pattern similar across facets (no strong interaction)")

In [ ]:
# Facet grid with regression lines
print("\n📊 FACET GRID WITH REGRESSION LINES")
print("="*60)

g = sns.lmplot(data=tips, x='total_bill', y='tip', col='time', row='sex', height=4, aspect=1.2)
g.fig.suptitle('Bill vs Tip: Regression by Time and Sex', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

print("\n💡 Insight:")
print("  - All groups show positive bill-tip relationship")
print("  - Similar slopes across groups")
print("  - No strong interaction between time and sex")

---

## Part 6: Detecting Interaction Effects 🔗

**Goal:** Find when variables work together

**Dataset:** Create data with known interaction

In [ ]:
# Create dataset with interaction
np.random.seed(42)
n = 400

interaction_data = pd.DataFrame({
    'experience_years': np.random.randint(0, 20, n),
    'training_hours': np.random.randint(0, 100, n),
    'department': np.random.choice(['Sales', 'Engineering', 'Marketing'], n)
})

# Create performance with interaction: training helps more for experienced workers
interaction_data['performance'] = (
    50 +  # Base
    2 * interaction_data['experience_years'] +  # Experience effect
    0.3 * interaction_data['training_hours'] +  # Training effect
    0.1 * interaction_data['experience_years'] * interaction_data['training_hours'] +  # INTERACTION!
    np.random.normal(0, 5, n)
)

print("Employee Performance Dataset (with interaction):")
print(interaction_data.head(10))
print(f"\nShape: {interaction_data.shape}")

In [ ]:
# Method 1: Visual detection - scatter with binned third variable
print("🔍 METHOD 1: VISUAL INTERACTION DETECTION")
print("="*60)

# Create experience level bins
interaction_data['exp_level'] = pd.cut(interaction_data['experience_years'], 
                                       bins=[0, 5, 10, 20], 
                                       labels=['Low (0-5)', 'Medium (6-10)', 'High (11-20)'])

plt.figure(figsize=(14, 5))

# Plot 1: Training effect at different experience levels
plt.subplot(1, 2, 1)
for level in ['Low (0-5)', 'Medium (6-10)', 'High (11-20)']:
    subset = interaction_data[interaction_data['exp_level'] == level]
    plt.scatter(subset['training_hours'], subset['performance'], alpha=0.5, label=level, s=30)
    
    # Add trend line
    if len(subset) > 10:
        z = np.polyfit(subset['training_hours'], subset['performance'], 1)
        p = np.poly1d(z)
        x_line = np.linspace(subset['training_hours'].min(), subset['training_hours'].max(), 100)
        plt.plot(x_line, p(x_line), linewidth=2.5)

plt.xlabel('Training Hours', fontsize=12)
plt.ylabel('Performance Score', fontsize=12)
plt.title('Training Effect by Experience Level\n(Different slopes = Interaction)', 
         fontsize=13, fontweight='bold')
plt.legend(title='Experience')
plt.grid(True, alpha=0.3)

# Plot 2: Experience effect at different training levels
plt.subplot(1, 2, 2)
interaction_data['training_level'] = pd.cut(interaction_data['training_hours'], 
                                           bins=[0, 33, 66, 100], 
                                           labels=['Low', 'Medium', 'High'])

for level in ['Low', 'Medium', 'High']:
    subset = interaction_data[interaction_data['training_level'] == level]
    plt.scatter(subset['experience_years'], subset['performance'], alpha=0.5, label=level, s=30)
    
    if len(subset) > 10:
        z = np.polyfit(subset['experience_years'], subset['performance'], 1)
        p = np.poly1d(z)
        x_line = np.linspace(subset['experience_years'].min(), subset['experience_years'].max(), 100)
        plt.plot(x_line, p(x_line), linewidth=2.5)

plt.xlabel('Experience (years)', fontsize=12)
plt.ylabel('Performance Score', fontsize=12)
plt.title('Experience Effect by Training Level\n(Different slopes = Interaction)', 
         fontsize=13, fontweight='bold')
plt.legend(title='Training')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🔍 Visual Evidence of Interaction:")
print("  ✓ LEFT: Training effect is stronger for high-experience workers (steeper slope)")
print("  ✓ RIGHT: Experience effect is stronger for high-training workers (steeper slope)")
print("  ✓ Non-parallel lines = INTERACTION EXISTS!")

In [ ]:
# Method 2: Statistical detection - correlation changes
print("\n📊 METHOD 2: CORRELATION-BASED DETECTION")
print("="*60)

print("\nCorrelation of Training with Performance by Experience Level:")
print("-" * 60)

for level in ['Low (0-5)', 'Medium (6-10)', 'High (11-20)']:
    subset = interaction_data[interaction_data['exp_level'] == level]
    corr, p_value = pearsonr(subset['training_hours'], subset['performance'])
    print(f"{level:20s}: r = {corr:+.3f} (p = {p_value:.4f})")

print("\n💡 Interpretation:")
print("  If correlations DIFFER significantly across groups → Interaction exists")
print("  Here: Correlation increases with experience level → INTERACTION CONFIRMED")

In [ ]:
# Method 3: Create interaction feature
print("\n🔗 METHOD 3: INTERACTION FEATURE")
print("="*60)

# Create interaction term
interaction_data['exp_x_training'] = interaction_data['experience_years'] * interaction_data['training_hours']

# Compare correlations
corr_exp, _ = pearsonr(interaction_data['experience_years'], interaction_data['performance'])
corr_training, _ = pearsonr(interaction_data['training_hours'], interaction_data['performance'])
corr_interaction, _ = pearsonr(interaction_data['exp_x_training'], interaction_data['performance'])

print("\nFeature Correlations with Performance:")
print("-" * 60)
print(f"Experience alone:         r = {corr_exp:+.4f}")
print(f"Training alone:           r = {corr_training:+.4f}")
print(f"Experience × Training:    r = {corr_interaction:+.4f}")

print("\n💡 Conclusion:")
if abs(corr_interaction) > max(abs(corr_exp), abs(corr_training)):
    print("  ✅ STRONG INTERACTION DETECTED!")
    print("  The interaction term has the strongest correlation")
    print("  Recommendation: Include interaction feature in modeling")
else:
    print("  ❌ No strong interaction")
    print("  Individual features sufficient")

---

## Part 7: Multivariate Outlier Detection 🎯

**Goal:** Find points unusual in combination

In [ ]:
# Create dataset with multivariate outliers
np.random.seed(100)
n_normal = 200
n_outliers = 10

# Normal data
normal_data = np.random.multivariate_normal(
    mean=[50, 50, 50], 
    cov=[[100, 50, 30], [50, 100, 40], [30, 40, 100]], 
    size=n_normal
)

# Outliers (individually normal range, but unusual combination)
outlier_data = np.array([
    [45, 48, 80],  # High on 3rd dimension given 1st and 2nd
    [52, 55, 20],  # Low on 3rd dimension
    [30, 70, 50],  # Unusual combination
    [70, 30, 50],  # Unusual combination
    [50, 50, 90],  # Very high on 3rd
    [48, 52, 15],  # Very low on 3rd
    [25, 75, 48],  # Extreme on 1st and 2nd
    [75, 25, 52],  # Extreme on 1st and 2nd
    [42, 42, 85],  # High on 3rd
    [58, 58, 18],  # Low on 3rd
])

# Combine
all_data = np.vstack([normal_data, outlier_data])
outlier_df = pd.DataFrame(all_data, columns=['Feature1', 'Feature2', 'Feature3'])
outlier_df['is_outlier'] = ['Normal'] * n_normal + ['Outlier'] * n_outliers

print("Dataset with Multivariate Outliers:")
print(outlier_df.head())
print(f"\nShape: {outlier_df.shape}")
print(f"\nOutliers: {n_outliers} / {len(outlier_df)} ({n_outliers/len(outlier_df)*100:.1f}%)")

In [ ]:
# Method 1: Isolation Forest
print("🌲 METHOD 1: ISOLATION FOREST")
print("="*60)

from sklearn.ensemble import IsolationForest

# Fit model
iso_forest = IsolationForest(contamination=0.05, random_state=42)
predictions = iso_forest.fit_predict(outlier_df[['Feature1', 'Feature2', 'Feature3']])

# -1 for outliers, 1 for inliers
outlier_df['iso_forest_pred'] = ['Outlier' if p == -1 else 'Normal' for p in predictions]

# Evaluate
from sklearn.metrics import classification_report, confusion_matrix

print("\nConfusion Matrix:")
cm = confusion_matrix(outlier_df['is_outlier'], outlier_df['iso_forest_pred'], labels=['Normal', 'Outlier'])
print(cm)

detected = (outlier_df['is_outlier'] == 'Outlier') & (outlier_df['iso_forest_pred'] == 'Outlier')
print(f"\nOutliers detected: {detected.sum()} / {n_outliers} ({detected.sum()/n_outliers*100:.1f}%)")

In [ ]:
# Visualize outlier detection
print("\n📊 VISUALIZATION OF DETECTED OUTLIERS")
print("="*60)

fig = plt.figure(figsize=(14, 5))

# Plot 1: Feature 1 vs Feature 2
ax1 = fig.add_subplot(131)
for label, color, marker in [('Normal', 'blue', 'o'), ('Outlier', 'red', 'X')]:
    subset = outlier_df[outlier_df['is_outlier'] == label]
    ax1.scatter(subset['Feature1'], subset['Feature2'], 
               c=color, marker=marker, s=100 if label == 'Outlier' else 30,
               alpha=0.6, label=f'True {label}')
ax1.set_xlabel('Feature 1', fontsize=11)
ax1.set_ylabel('Feature 2', fontsize=11)
ax1.set_title('Feature 1 vs 2', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Feature 1 vs Feature 3
ax2 = fig.add_subplot(132)
for label, color, marker in [('Normal', 'blue', 'o'), ('Outlier', 'red', 'X')]:
    subset = outlier_df[outlier_df['is_outlier'] == label]
    ax2.scatter(subset['Feature1'], subset['Feature3'], 
               c=color, marker=marker, s=100 if label == 'Outlier' else 30,
               alpha=0.6, label=f'True {label}')
ax2.set_xlabel('Feature 1', fontsize=11)
ax2.set_ylabel('Feature 3', fontsize=11)
ax2.set_title('Feature 1 vs 3', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: 3D view
ax3 = fig.add_subplot(133, projection='3d')
for label, color, marker in [('Normal', 'blue', 'o'), ('Outlier', 'red', 'X')]:
    subset = outlier_df[outlier_df['is_outlier'] == label]
    ax3.scatter(subset['Feature1'], subset['Feature2'], subset['Feature3'],
               c=color, marker=marker, s=100 if label == 'Outlier' else 30,
               alpha=0.6, label=f'True {label}')
ax3.set_xlabel('Feature 1', fontsize=10)
ax3.set_ylabel('Feature 2', fontsize=10)
ax3.set_zlabel('Feature 3', fontsize=10)
ax3.set_title('3D View', fontsize=12, fontweight='bold')
ax3.legend()

plt.tight_layout()
plt.show()

print("\n💡 Key Insight:")
print("  - Outliers may look normal in 2D projections")
print("  - Only unusual when considering all dimensions together")
print("  - This is why multivariate methods are necessary!")

---

## Part 8: Dimensionality Reduction (PCA & t-SNE) 🗜️

**Goal:** Visualize high-dimensional data in 2D

In [ ]:
# Use Iris dataset (4 features)
print("🗜️ DIMENSIONALITY REDUCTION")
print("="*60)

# Prepare features
X = iris.drop('species', axis=1)
y = iris['species']

print(f"\nOriginal data: {X.shape[1]} dimensions (features)")
print(f"Goal: Reduce to 2 dimensions for visualization")
print(f"Samples: {X.shape[0]}")

In [ ]:
# Method 1: PCA (Principal Component Analysis)
print("\n📊 METHOD 1: PCA (Principal Component Analysis)")
print("="*60)

from sklearn.decomposition import PCA

# Standardize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

print(f"\n✅ Reduced from {X.shape[1]}D to 2D")
print(f"\nVariance explained by components:")
print(f"  PC1: {pca.explained_variance_ratio_[0]*100:.1f}%")
print(f"  PC2: {pca.explained_variance_ratio_[1]*100:.1f}%")
print(f"  Total: {pca.explained_variance_ratio_.sum()*100:.1f}%")

# Visualize PCA
plt.figure(figsize=(10, 6))
colors_map = {'setosa': 'red', 'versicolor': 'green', 'virginica': 'blue'}
for species in y.unique():
    mask = y == species
    plt.scatter(X_pca[mask, 0], X_pca[mask, 1], 
               c=colors_map[species], label=species, alpha=0.6, s=50)

plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)', fontsize=12)
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)', fontsize=12)
plt.title('PCA: Iris Dataset (4D → 2D)', fontsize=14, fontweight='bold')
plt.legend(title='Species')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 Observations:")
print("  ✓ Setosa clearly separated (bottom left)")
print("  ✓ Versicolor and Virginica partially overlap")
print("  ✓ First 2 components capture 95.8% of variance")
print("  ✓ Linear projection preserves global structure")

In [ ]:
# Method 2: t-SNE
print("\n📊 METHOD 2: t-SNE (t-Distributed Stochastic Neighbor Embedding)")
print("="*60)

from sklearn.manifold import TSNE

# Apply t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_scaled)

print(f"\n✅ Reduced from {X.shape[1]}D to 2D using t-SNE")
print(f"Parameters: perplexity=30, random_state=42")

# Visualize t-SNE
plt.figure(figsize=(10, 6))
for species in y.unique():
    mask = y == species
    plt.scatter(X_tsne[mask, 0], X_tsne[mask, 1], 
               c=colors_map[species], label=species, alpha=0.6, s=50)

plt.xlabel('t-SNE Component 1', fontsize=12)
plt.ylabel('t-SNE Component 2', fontsize=12)
plt.title('t-SNE: Iris Dataset (4D → 2D)', fontsize=14, fontweight='bold')
plt.legend(title='Species')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 Observations:")
print("  ✓ ALL three species well-separated")
print("  ✓ Tighter clusters than PCA")
print("  ✓ Better local structure preservation")
print("  ✓ Note: Distances between clusters not meaningful!")

In [ ]:
# Compare PCA vs t-SNE side by side
print("\n📊 COMPARISON: PCA vs t-SNE")
print("="*60)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# PCA plot
for species in y.unique():
    mask = y == species
    axes[0].scatter(X_pca[mask, 0], X_pca[mask, 1], 
                   c=colors_map[species], label=species, alpha=0.6, s=50)
axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)', fontsize=11)
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)', fontsize=11)
axes[0].set_title('PCA (Linear, Fast, Global Structure)', fontsize=12, fontweight='bold')
axes[0].legend(title='Species')
axes[0].grid(True, alpha=0.3)

# t-SNE plot
for species in y.unique():
    mask = y == species
    axes[1].scatter(X_tsne[mask, 0], X_tsne[mask, 1], 
                   c=colors_map[species], label=species, alpha=0.6, s=50)
axes[1].set_xlabel('t-SNE Component 1', fontsize=11)
axes[1].set_ylabel('t-SNE Component 2', fontsize=11)
axes[1].set_title('t-SNE (Non-linear, Slow, Local Structure)', fontsize=12, fontweight='bold')
axes[1].legend(title='Species')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 PCA vs t-SNE Summary:")
print("-" * 60)
print("PCA:")
print("  ✓ Fast computation")
print("  ✓ Preserves global structure")
print("  ✓ Variance explained is interpretable")
print("  ✓ Deterministic (same result every time)")
print("  ✗ Only linear transformations")

print("\nt-SNE:")
print("  ✓ Excellent cluster separation")
print("  ✓ Preserves local structure")
print("  ✓ Can capture non-linear patterns")
print("  ✗ Slow for large datasets")
print("  ✗ Different runs give different results")
print("  ✗ Distances between clusters not meaningful")

---

## Part 9: Complete Multivariate Analysis Workflow 🔄

**Goal:** Analyze a complex dataset using all techniques

In [ ]:
# Load comprehensive dataset
print("🔄 COMPLETE MULTIVARIATE ANALYSIS WORKFLOW")
print("="*70)

# Use penguins dataset
penguins_clean = penguins.copy()

print(f"\nDataset: Palmer Penguins")
print(f"Samples: {len(penguins_clean)}")
print(f"Features: {penguins_clean.select_dtypes(include=[np.number]).columns.tolist()}")
print(f"Categories: species ({penguins_clean['species'].nunique()}), island ({penguins_clean['island'].nunique()})")

In [ ]:
# Step 1: Correlation heatmap
print("\n📊 STEP 1: CORRELATION OVERVIEW")
print("="*70)

numerical_features = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
corr_matrix = penguins_clean[numerical_features].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1)
plt.title('Penguin Features Correlation Matrix', fontsize=14, fontweight='bold', pad=15)
plt.tight_layout()
plt.show()

print("\n🔍 Key Findings:")
print("  ✓ Flipper length & body mass: Strong positive (r=0.87)")
print("  ✓ Bill length & flipper length: Moderate positive (r=0.66)")
print("  ✓ Bill depth negatively correlated with others (Simpson's Paradox candidate)")

In [ ]:
# Step 2: Pair plot with species
print("\n📊 STEP 2: PAIR PLOT ANALYSIS")
print("="*70)

sns.pairplot(penguins_clean, vars=numerical_features, hue='species', 
            palette='Set1', diag_kind='kde', corner=True)
plt.suptitle('Penguin Features - All Pairwise Relationships', y=1.02, fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n🔍 Key Findings:")
print("  ✓ Clear species separation on flipper_length & body_mass")
print("  ✓ Bill measurements show species-specific patterns")
print("  ✓ Gentoo: Largest (long flippers, heavy)")
print("  ✓ Adelie: Smallest, deep bills")
print("  ✓ Chinstrap: Medium, long thin bills")

In [ ]:
# Step 3: Check for interactions
print("\n📊 STEP 3: INTERACTION ANALYSIS")
print("="*70)

# Does bill length-body mass relationship vary by species?
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, species_name in enumerate(['Adelie', 'Gentoo', 'Chinstrap']):
    data = penguins_clean[penguins_clean['species'] == species_name]
    axes[idx].scatter(data['bill_length_mm'], data['body_mass_g'], alpha=0.6, s=50)
    
    # Add regression line
    z = np.polyfit(data['bill_length_mm'], data['body_mass_g'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(data['bill_length_mm'].min(), data['bill_length_mm'].max(), 100)
    axes[idx].plot(x_line, p(x_line), 'r-', linewidth=2)
    
    # Calculate correlation
    corr, _ = pearsonr(data['bill_length_mm'], data['body_mass_g'])
    
    axes[idx].set_xlabel('Bill Length (mm)', fontsize=11)
    axes[idx].set_ylabel('Body Mass (g)', fontsize=11)
    axes[idx].set_title(f'{species_name}\nr = {corr:.3f}', fontsize=12, fontweight='bold')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🔍 Interaction Detection:")
print("  ✓ Different slopes across species")
print("  ✓ Species moderates the bill-mass relationship")
print("  ✓ This is an INTERACTION effect")

In [ ]:
# Step 4: Dimensionality reduction for overview
print("\n📊 STEP 4: DIMENSIONALITY REDUCTION (PCA)")
print("="*70)

# Prepare data
X_penguins = penguins_clean[numerical_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_penguins)

# PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Plot
plt.figure(figsize=(10, 6))
colors_map = {'Adelie': '#E74C3C', 'Gentoo': '#2ECC71', 'Chinstrap': '#3498DB'}
for species in penguins_clean['species'].unique():
    mask = penguins_clean['species'] == species
    plt.scatter(X_pca[mask, 0], X_pca[mask, 1], 
               c=colors_map[species], label=species, alpha=0.6, s=50)

plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)', fontsize=12)
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)', fontsize=12)
plt.title(f'PCA: Penguins Overview\n(Captures {pca.explained_variance_ratio_.sum()*100:.1f}% variance)', 
         fontsize=14, fontweight='bold')
plt.legend(title='Species')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n✅ PCA captures {pca.explained_variance_ratio_.sum()*100:.1f}% of variance in 2D")
print("  ✓ Clear species separation")
print("  ✓ Gentoo distinct on PC1")
print("  ✓ Adelie and Chinstrap separated on PC2")

In [ ]:
# Step 5: Summary
print("\n" + "="*70)
print("📋 MULTIVARIATE ANALYSIS SUMMARY")
print("="*70)

print("\n1️⃣ CORRELATION ANALYSIS:")
print("   ✓ Strong correlation: flipper_length ↔ body_mass (r=0.87)")
print("   ✓ Moderate correlation: bill_length ↔ flipper_length (r=0.66)")
print("   ✓ Negative correlation: bill_depth ↔ others (species effect)")

print("\n2️⃣ PAIRWISE RELATIONSHIPS:")
print("   ✓ Three distinct species clusters")
print("   ✓ Best separators: flipper_length, body_mass")
print("   ✓ Species-specific bill shapes")

print("\n3️⃣ INTERACTIONS:")
print("   ✓ Species moderates bill-mass relationship")
print("   ✓ Different correlations within each species")
print("   ✓ Overall negative correlation becomes positive within species")

print("\n4️⃣ DIMENSIONALITY REDUCTION:")
print(f"   ✓ 2 components capture {pca.explained_variance_ratio_.sum()*100:.1f}% variance")
print("   ✓ Clear species separation in reduced space")
print("   ✓ Original 4D structure well-preserved in 2D")

print("\n💡 OVERALL INSIGHTS:")
print("   - Species is the primary driver of variation")
print("   - Physical measurements are highly correlated within species")
print("   - flipper_length and body_mass are best discriminators")
print("   - Multivariate view reveals Simpson's Paradox in bill depth")

print("\n✅ Complete multivariate analysis workflow executed!")

---

## 🎯 Key Takeaways

### Multivariate Analysis Workflow:

1. **Start with correlations** → Identify relationships
2. **Create pair plots** → See distributions and pairs
3. **Use parallel coordinates** → Spot patterns
4. **Check for interactions** → Look for conditional relationships
5. **Apply dimensionality reduction** → Get overview
6. **Detect multivariate outliers** → Find unusual combinations
7. **Create facet grids** → Compare across categories

### Visualization Cheat Sheet:

| Goal | Use | When |
|------|-----|------|
| Overview all features | Pair plot | 4-10 numerical vars |
| See all correlations | Heatmap | Any number of vars |
| Compare patterns | Parallel coordinates | 5-15 vars |
| Visualize 3 vars | 3D scatter | Exactly 3 numerical |
| Show 4 vars | Bubble chart | 2 numerical + 1 size + 1 color |
| Compare groups | Facet grid | Conditional relationships |
| Find clusters (many vars) | PCA/t-SNE | 10+ vars |
| Detect outliers | Isolation Forest | Unusual combinations |

---

## ✅ What You've Learned

1. ✅ Create and interpret pair plots
2. ✅ Build correlation heatmaps
3. ✅ Use parallel coordinates plots
4. ✅ Create 3D visualizations and bubble charts
5. ✅ Build facet grids for comparisons
6. ✅ Detect interaction effects
7. ✅ Identify multivariate outliers
8. ✅ Apply PCA and t-SNE
9. ✅ Execute complete multivariate workflow

---

## 🎓 Next Steps

Ready to practice? Move on to **12_exercises.ipynb** to:
- Apply these techniques to new datasets
- Practice multivariate analysis
- Build comprehensive analysis skills

**Happy Analyzing! 📊🔍✨**